Input Required: AutoCAD Dxf drawing that has multiple lines connecting each other and forming geometric pattern of triangles, quadrilaterals.

Output: AutoCAD Dxf drawing with 3DFace entity (a three-sided or four-sided surface in 3D space)

Note: As a 3DFace can have clockwise or anti-clockwise direction, user has to draw only one 3DFace in the input drawing, so that algorithm will refer that 3DFace and generate all the remaining 3DFaces in the same direction.

Usage:

    1. As this converts the edges into surface, this can be used and further enhanced to identify the painting area or can derive the number/size of steel plates/glasses that is going to fit into that.

    2. Getting 3DFace entity can also be an intermediate step for getting manageable amount of small and detailed drawing from full model.

Sincere thanks to Pradeep Hebbar for bringing up the idea of this automation work since we had seen the complexity and time consumption of manually drawing up the 3DFace from line geometry.

In [60]:
# Set the AutoCAD input file path
InputDxfFilepath = r'E:\AutoCAD-analytical-geometry\AutoCAD_input_file.dxf'
OutputDxfFilepath = r'E:\AutoCAD-analytical-geometry\AutoCAD_output_file.dxf'
RoundToDecimal = 2

In [61]:
# pip install ezdxf

In [62]:
# Import the required python libraries
import sys
import ezdxf
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [63]:
# ThreeDFace class to hold list of 3 or 4 geometry points (NodeIds).
# i.e, NodeIds = [[x1, y1, z1], [x2, y2, z2], [x3, y3, z3]]
# or NodeIds = [[x1, y1, z1], [x2, y2, z2], [x3, y3, z3], [x4, y4, z4]]

# ThreeDFaceBuilder class will hold NodeIds and also will hold some properties and functions to build ThreeDFace.

class ThreeDFace:
    NodeIds = []
class ThreeDFaceBuilder:
    NodeIds = []
    def __GetLastPnt(self):
        return self.NodeIds[-1] if len(self.NodeIds) > 0 else ''
    LastPnt = property(__GetLastPnt)
    def __PreLastPnt(self):
        return self.NodeIds[-2] if len(self.NodeIds) > 1 else ''
    PreLastPnt = property(__PreLastPnt)
    def __GetFaceComplete(self):
        return self.NodeIds[0] == self.NodeIds[-1] if len(self.NodeIds) >= 4 else False 
    FaceComplete = property(__GetFaceComplete)
    def AddNodeId(self, NodeId):
        if self.FaceComplete == False:
            self.NodeIds.append(NodeId)

def GetPreFacePoint(ThreeDFace: ThreeDFace, FaceNode: int):
    PreNode = FaceNode - 1
    return ThreeDFace.NodeIds[-2] if PreNode == -1 else ThreeDFace.NodeIds[PreNode]

def IsSameLine(Line1, Line2):
    return (Line1[0] == Line2[0]) & (Line1[1] == Line2[1]) | (Line1[0] == Line2[1]) & (Line1[1] == Line2[0])

def PointInLine(Line, Pnt):
    return Line.count(Pnt) > 0

def AnotherPointInLine(Line, Pnt):
    return Line[0] if Pnt == Line[1] else Line[1]

def GetLineId(node1_id: str, node2_id: str):
    return node1_id + '_' + node2_id if node1_id < node2_id else node2_id + '_' + node1_id

In [64]:
# Read AutoCAD file
try:
    AutocadDxfDoc = ezdxf.readfile(InputDxfFilepath)
except IOError:
    print(f'Not a DXF file or a generic I/O error.')
    sys.exit(1)
except ezdxf.DXFStructureError:
    print(f'Invalid or corrupted DXF file.')
    sys.exit(2)

In [65]:
# Extract lines from AutoCAD model.
ModelSpace = AutocadDxfDoc.modelspace()
Lines = ModelSpace.query('LINE')
P1List, P2List = [], []
for Line in Lines:
    P1List.append(Line.dxf.start)
    P2List.append(Line.dxf.end)

In [66]:
# Make a Pandas dataframe to hold the coordinates for the extracted lines.
LinesColumnCaptionList = ['x1', 'y1', 'z1', 'x2', 'y2', 'z2']
AllLinesDf = pd.concat([pd.DataFrame(P1List), pd.DataFrame(P2List)], axis = 1)
AllLinesDf.columns = LinesColumnCaptionList
AllLinesDf = pd.concat([AllLinesDf, pd.DataFrame(np.round(AllLinesDf, RoundToDecimal))], axis = 1)
for i in range(0, len(LinesColumnCaptionList)):
    AllLinesDf.columns.values[len(LinesColumnCaptionList) + i] = 'rounded_' + LinesColumnCaptionList[i]

In [67]:
# Make a Pandas dataframe to hold the coordinates for the two points of the extracted lines.
# Set node_id for unique identification of a point.

PointsColumnCaptionList = ['x', 'y', 'z', 'node_x', 'node_y', 'node_z']
P1List = AllLinesDf[['x1', 'y1', 'z1', 'rounded_x1', 'rounded_y1', 'rounded_z1']]
P2List = AllLinesDf[['x2', 'y2', 'z2', 'rounded_x2', 'rounded_y2', 'rounded_z2']]
AllPointsDf = pd.concat([pd.DataFrame(P1List[:].values), pd.DataFrame(P2List[:].values)], axis = 0)
AllPointsDf.columns = PointsColumnCaptionList
AllPointsDf = AllPointsDf.drop_duplicates(['node_x', 'node_y', 'node_z'], keep = 'first')
AllPointsDf = AllPointsDf.reset_index(drop=True)
AllPointsDf['node_id'] = AllPointsDf.index
AllPointsDf['node_id'] = AllPointsDf['node_id'].apply(lambda x: 'P' + str(x))

In [68]:
# Set line_id for unique identification of a line.
# Drop duplicate lines if any.
# For example if Line1 is start point is p1 and end point is p2, 
# if Line2 is start point is p2 and end point is p1 then both are duplicate lines and one of them can be dropped.

AllLinesDf = AllLinesDf.merge(AllPointsDf, how='left', left_on = ['rounded_x1', 'rounded_y1', 'rounded_z1'], \
                                                      right_on = ['node_x', 'node_y', 'node_z'])
AllLinesDf = AllLinesDf.drop(PointsColumnCaptionList, axis = 1)
AllLinesDf = AllLinesDf.rename(columns={'node_id' : 'node1_id'})

AllLinesDf = AllLinesDf.merge(AllPointsDf, how='left', left_on = ['rounded_x2', 'rounded_y2', 'rounded_z2'], \
                                                      right_on = ['node_x', 'node_y', 'node_z'])
AllLinesDf = AllLinesDf.drop(PointsColumnCaptionList, axis = 1)
AllLinesDf = AllLinesDf.rename(columns={'node_id' : 'node2_id'})

AllLinesDf['line_id'] = AllLinesDf.apply(lambda x: GetLineId(x['node1_id'], x['node2_id']), axis=1)
AllLinesDf = AllLinesDf.drop_duplicates('line_id')

In [69]:
# Fetch the given ThreeDFaceModelPoint from the input file.
# This sample 3DFace is used to determine the clockwise or anti-clockwise direction.
# The algorithm will refer this 3DFace and generate all the remaining 3DFaces in the same direction.

ThreeDFaceModelList =  ModelSpace.query('3DFACE')
ThreeDFaceModelPoint = []
ThreeDFaceModelPoint.append(np.round(ThreeDFaceModelList[0].dxf.vtx0, RoundToDecimal))
ThreeDFaceModelPoint.append(np.round(ThreeDFaceModelList[0].dxf.vtx1, RoundToDecimal))
ThreeDFaceModelPoint.append(np.round(ThreeDFaceModelList[0].dxf.vtx2, RoundToDecimal))
ThreeDFaceModelPoint.append(np.round(ThreeDFaceModelList[0].dxf.vtx3, RoundToDecimal))

In [70]:
# Get ThreeDFace class object from the ThreeDFaceModelPoint.
InputThreeDFace = ThreeDFace()
InputThreeDFace.NodeIds = []
for i in range(0, 4):
    aPoint = AllPointsDf[(AllPointsDf.node_x == ThreeDFaceModelPoint[i][0]) & \
                         (AllPointsDf.node_y == ThreeDFaceModelPoint[i][1]) & \
                         (AllPointsDf.node_z == ThreeDFaceModelPoint[i][2])]
    InputThreeDFace.NodeIds.append(aPoint.iloc[0].loc['node_id'])

In [71]:
# NodeOnlyLinesList is sub-set of AllLinesDf to hold only the node-ids not coordinates.
# DeletedLineList to hold DeletedLines. We keep deleting lines once we build corresponding ThreeDFace
# or otherwise, algorithm will keep building duplicate ThreeDFace.
# ThreeDFaceFinalList  to hold the output ThreeDFace list.

NodeOnlyLinesList = AllLinesDf[['node1_id', 'node2_id', 'line_id']]
DeletedLineList = []
ThreeDFaceFinalList = []

In [72]:
# Search algorithm that builds ThreeDFaces from the NodeIds.

def SearchThreeDFace(aThreeDFace: ThreeDFace):
    for FaceNode in range(0, len(aThreeDFace.NodeIds) - 1):
        p1 = aThreeDFace.NodeIds[FaceNode + 1]
        p2 = aThreeDFace.NodeIds[FaceNode]
        BaseLine  = [p1, p2]
        ThreeDFaceBuilderInit = ThreeDFaceBuilder()
        ThreeDFaceBuilderInit.NodeIds = []
        ThreeDFaceBuilderInit.NodeIds.append(p1)
        ThreeDFaceBuilderInit.NodeIds.append(p2)
        ThreeDFaceBuilderList: list[ThreeDFaceBuilder] = []
        ThreeDFaceBuilderList.append(ThreeDFaceBuilderInit)
        PreFacePoint = GetPreFacePoint(aThreeDFace, FaceNode)
        for i in range(0, 3):
            ThreeDFaceBuilderListNew = []
            for j in range(0, len(ThreeDFaceBuilderList)):
                aThreeDFaceBuilder = ThreeDFaceBuilderList[j]
                SubSetLineList = NodeOnlyLinesList[(NodeOnlyLinesList.node1_id == aThreeDFaceBuilder.LastPnt) | \
                                (NodeOnlyLinesList.node2_id == aThreeDFaceBuilder.LastPnt)][:].values
                PntBuilderList = []
                for CurrLine in SubSetLineList:
                     if (IsSameLine(CurrLine, [aThreeDFaceBuilder.LastPnt, aThreeDFaceBuilder.PreLastPnt]) == False) & \
                        (IsSameLine(CurrLine, [aThreeDFaceBuilder.LastPnt, PreFacePoint]) == False):
                        PntBuilderList.append(AnotherPointInLine(CurrLine, aThreeDFaceBuilder.LastPnt))
                for Pnt in PntBuilderList:
                    ThreeDFaceBuilderNew = ThreeDFaceBuilder()
                    ThreeDFaceBuilderNew.NodeIds = []
                    for BasePnt in aThreeDFaceBuilder.NodeIds:
                        ThreeDFaceBuilderNew.AddNodeId(BasePnt)
                    ThreeDFaceBuilderNew.AddNodeId(Pnt)   
                    ThreeDFaceBuilderListNew.append(ThreeDFaceBuilderNew)
            ThreeDFaceBuilderList = ThreeDFaceBuilderListNew
        ThreeDFaceBuilderList = filter(lambda x: x.FaceComplete == True, ThreeDFaceBuilderList)        
        ThreeDFaceBuilderList = sorted(ThreeDFaceBuilderList, key=lambda x:len(x.NodeIds))    
        if len(ThreeDFaceBuilderList) > 0:
            aThreeDFaceBuilder = ThreeDFaceBuilderList[0]
            DeletedLine = False
            for DeletedLines in DeletedLineList:
                if (IsSameLine([aThreeDFaceBuilder.NodeIds[0], aThreeDFaceBuilder.NodeIds[2]], DeletedLines) == True) | \
                   (IsSameLine([aThreeDFaceBuilder.NodeIds[1], aThreeDFaceBuilder.NodeIds[3]], DeletedLines) == True):
                    DeletedLine = True
            if DeletedLine ==  False:
                NewThreedFace = ThreeDFace()
                NewThreedFace.NodeIds = aThreeDFaceBuilder.NodeIds
                ThreeDFaceFinalList.append(NewThreedFace)
                DeletedLineList.append(BaseLine)
                index_to_delete = NodeOnlyLinesList[(NodeOnlyLinesList['line_id'] == GetLineId(BaseLine[0], BaseLine[1]))].index
                NodeOnlyLinesList.drop(index_to_delete, inplace = True)

In [73]:
# Add the InputThreeDFace as well to the output.
ThreeDFaceFinalList.append(InputThreeDFace)
for bThreedFace in ThreeDFaceFinalList:
    SearchThreeDFace(bThreedFace)

In [74]:
# Save AutoCAD output file with ThreeDFaces.
doc = ezdxf.new('R2010')  # create a new DXF R2010 drawing, official DXF version name: 'AC1024'
msp = doc.modelspace()  # add new entities to the modelspace
cThreeDFace = ThreeDFace()
for cThree3Face in ThreeDFaceFinalList:
    Vertex = []
    for k in range(0, 4):
        p = AllPointsDf[AllPointsDf.node_id == cThree3Face.NodeIds[k]][['x', 'y', 'z']].values
        Vertex.append(tuple(p[0])) 
    msp.add_3dface(Vertex)
doc.saveas(OutputDxfFilepath)